In [1]:
# mount drive for access to the files
from google.colab import drive

drive.mount("/content/drive")

# all the drive the files are present in "/content/drive/My Drive"
!ls "/content/drive/My Drive/Beuth Uni/Master Thesis"

import sys
sys.path.append('/content/drive/My Drive/Beuth Uni/Master Thesis/jenga')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Data
'Data Quality in ML Production Systems.pdf'
'Datawig: Missing Value Imputation for Tables.pdf'
 Declaration
 Images
 jenga
 jenga.pdf
 MICE_Multivariate_Imputation_by_Chained_Equations_.pdf


In [2]:
!pip install openml
!pip install pyod

!pip install mxnet autogluon
!pip install mxnet-mkl --pre --upgrade

Requirement already up-to-date: mxnet-mkl in /usr/local/lib/python3.6/dist-packages (1.6.0)


In [3]:
import random
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from jenga.basis import Dataset
from jenga.corruptions.generic import MissingValues, SwappedValues
from jenga.corruptions.numerical import Scaling, GaussianNoise
from jenga.cleaning.ppp import PipelinePerformancePrediction
from jenga.cleaning.outlier_detection import NoOutlierDetection, PyODKNN, PyODIsolationForest
from jenga.cleaning.imputation import NoImputation, MeanModeImputation, AutoGluonImputation
from jenga.cleaning.clean import Clean

In [4]:
seed = 10

In [5]:
def run_experiment(dataset_name, learner, param_grid, corruptions, fraction, cleaners, num_repetitions):
    
    ## dataset
    dataset = Dataset(seed, dataset_name)
    
    all_data = dataset.all_data
    attribute_names = dataset.attribute_names
    attribute_types = dataset.attribute_types
    
    ## categorical and numerical features
    categorical_columns = dataset.categorical_columns
    numerical_columns = dataset.numerical_columns
    print(f"Found {len(categorical_columns)} categorical and {len(numerical_columns)} numeric features \n")
    
    ## train and test data
    df_train, lab_train, df_test, lab_test = dataset.get_train_test_data()
    
    
    ## pipeline performance prediction (ppp)
    ppp = PipelinePerformancePrediction(seed, df_train, lab_train, df_test, lab_test, categorical_columns, numerical_columns, learner, param_grid)
    ppp_model = ppp.fit_ppp(df_train)
    
    ## generate corrpted data
    df_corrupted, perturbations, cols_perturbed, summary_col_corrupt = ppp.get_corrupted(df_test, corruptions, fraction, num_repetitions)
    
    ## cleaning
    clean = Clean(df_train, df_corrupted, categorical_columns, numerical_columns, ppp, ppp_model, cleaners)
    df_cleaned, corrupted_score_ppp, best_cleaning_score, cleaner_scores_ppp, summary_cleaners = clean(df_train, df_corrupted)
    
    ## results
    result = {
        'ppp_score_model': ppp.predict_score_ppp(ppp_model, df_test),
        'ppp_score_corrupted': corrupted_score_ppp,
        'ppp_score_cleaned': best_cleaning_score,
        'ppp_scores_cleaners': cleaner_scores_ppp
    }
#     print('\n'.join([f'{key}:{val}' for key, val in result.items()]))
    
    ## summary
    summary = {
        'dataset': dataset_name,
        'model': learner,
        'corruptions': summary_col_corrupt,
        'cleaners': summary_cleaners,
        'result': result
    }
#     print('\n\n\n\n'.join([f'{key}:{val}' for key, val in summary.items()]))
    
    return summary #summary_col_corrupt, result

In [6]:
datasets = [
    'parkinsons',
    'heart-statlog',
    'credit-g'
]

In [7]:
## model parameters
## models is a dict where key = leaner & value = param_grid
models = {SGDClassifier(loss='log'): {'learner__max_iter': [500, 1000, 5000], 
                                         'learner__penalty': ['l2', 'l1', 'elasticnet'], 
                                         'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
                                        }, 
          RandomForestClassifier():{'learner__n_estimators': [100, 200, 500], 
                                    'learner__max_depth': [5, 10, 15]
                                   }
         }

## make dict of multiple leraners and corresponding param_grids

In [8]:
corruptions = [MissingValues, SwappedValues, Scaling, GaussianNoise]

In [9]:
fractions = np.random.uniform(0, 1, 1) # np.random.uniform(0, 1, 3)

In [10]:
cleaners = [
    (NoOutlierDetection, MeanModeImputation),
    (NoOutlierDetection, AutoGluonImputation),
    (PyODKNN, NoImputation),
    (PyODKNN, MeanModeImputation),
    (PyODKNN, AutoGluonImputation),
    (PyODIsolationForest, NoImputation),
    (PyODIsolationForest, MeanModeImputation),
    (PyODIsolationForest, AutoGluonImputation)
]

In [11]:
%%time
for _ in range(2):
  print("\n\n..................................ITERATION..................................\n")
  ind_results = []

  for dataset in datasets:
      for learner, param_grid in models.items():
          for fraction in fractions:
              ind_results.append(run_experiment(dataset, learner, param_grid, [MissingValues], fraction, [(PyODKNN, AutoGluonImputation)], 5))



..................................ITERATION..................................



Saved dataset 1488: parkinsons to file /root/.openml/cache/org/openml/www/datasets/1488/dataset.pkl.py3


Dataset: parkinsons
Found 0 categorical and 22 numeric features 

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    3.0s finished



Generating corrupted training data on 39 rows... 

	perturbation: MissingValues: {'column': 'V2', 'fraction': 0.426920161499486, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'V16', 'fraction': 0.426920161499486, 'na_value': nan, 'missingness': 'MAR'}
	perturbation: MissingValues: {'column': 'V1', 'fraction': 0.426920161499486, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'V15', 'fraction': 0.426920161499486, 'na_value': nan, 'missingness': 'MCAR'}
	perturbation: MissingValues: {'column': 'V9', 'fraction': 0.426920161499486, 'na_value': nan, 'missingness': 'MAR'}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.9837662337662337, 'classification_report': {'1': {'precision': 0.6875, 'recall': 1.0, 'f1-score': 0.8148148148148148, 'support': 11}, '2': {'precision': 1.0, 'recall': 0.8214285714285714, 'f1-score': 0.9019607843137255, 'support': 28}, 'accuracy': 0.8717948717948718, 'macro avg': {'precisio

...


Generating corrupted training data on 200 rows... 

	perturbation: MissingValues: {'column': 'credit_amount', 'fraction': 0.426920161499486, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'other_payment_plans', 'fraction': 0.426920161499486, 'na_value': nan, 'missingness': 'MAR'}
	perturbation: MissingValues: {'column': 'duration', 'fraction': 0.426920161499486, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'property_magnitude', 'fraction': 0.426920161499486, 'na_value': nan, 'missingness': 'MCAR'}
	perturbation: MissingValues: {'column': 'credit_history', 'fraction': 0.426920161499486, 'na_value': nan, 'missingness': 'MAR'}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.7823749415614774, 'classification_report': {'bad': {'precision': 0.7096774193548387, 'recall': 0.3548387096774194, 'f1-score': 0.4731182795698926, 'support': 62}, 'good': {'precision': 0.7633136094674556, 'recall': 0.9347826086956

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200728_113613/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200728_113613/
AutoGluon Version:  0.0.12
Train Data Rows:    800
Train Data Columns: 20
Preprocessing data ...
Train Data Class Count: 4
Feature Generator processed 800 data points with 19 features
Original Features (raw dtypes):
	float64 features: 7
	object features: 12
Original Features (inferred dtypes):
	float features: 7
	object features: 12
Generated Features (special dtypes):
Final Features (raw dtypes):
	float features: 7
	category features: 12
Final Features:
	float features: 7
	category features: 12
	Data preprocessing and feature engineering runtime = 0.11s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: accuracy
Fitting model: RandomForestClassifierGini

Cleaner: {'outlier_detection': PyODKNN, 'imputation': AutoGluonImputation}: {'roc_auc_acore': 0.7823749415614774, 'classification_report': {'bad': {'precision': 0.7096774193548387, 'recall': 0.3548387096774194, 'f1-score': 0.4731182795698926, 'support': 62}, 'good': {'precision': 0.7633136094674556, 'recall': 0.9347826086956522, 'f1-score': 0.8403908794788274, 'support': 138}, 'accuracy': 0.755, 'macro avg': {'precision': 0.7364955144111471, 'recall': 0.6448106591865358, 'f1-score': 0.65675457952436, 'support': 200}, 'weighted avg': {'precision': 0.7466863905325444, 'recall': 0.755, 'f1-score': 0.7265363735070577, 'support': 200}}}


No output_directory specified. Models will be saved in: AutogluonModels/ag-20200728_114306/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200728_114306/
AutoGluon Version:  0.0.12
Train Data Rows:    800
Train Data Columns: 20
Preprocessing data ...
Train Data Class Count: 4
Feature Generator processed 800 data points with 19 features
Original Features (raw dtypes):
	float64 features: 7
	object features: 12
Original Features (inferred dtypes):
	float features: 7
	object features: 12
Generated Features (special dtypes):
Final Features (raw dtypes):
	float features: 7
	category features: 12
Final Features:
	float features: 7
	category features: 12
	Data preprocessing and feature engineering runtime = 0.1s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: accuracy
Fitting model: RandomForestClassifierGini 


Best cleaning method:
Cleaning score: Cleaner: {'outlier_detection': PyODKNN, 'imputation': AutoGluonImputation}: {'roc_auc_acore': 0.7823749415614774, 'classification_report': {'bad': {'precision': 0.7096774193548387, 'recall': 0.3548387096774194, 'f1-score': 0.4731182795698926, 'support': 62}, 'good': {'precision': 0.7633136094674556, 'recall': 0.9347826086956522, 'f1-score': 0.8403908794788274, 'support': 138}, 'accuracy': 0.755, 'macro avg': {'precision': 0.7364955144111471, 'recall': 0.6448106591865358, 'f1-score': 0.65675457952436, 'support': 200}, 'weighted avg': {'precision': 0.7466863905325444, 'recall': 0.755, 'f1-score': 0.7265363735070577, 'support': 200}}} 

Cleaning didnt't improve the overall score 



CPU times: user 1h 48min 14s, sys: 5min 43s, total: 1h 53min 58s
Wall time: 1h 27min 53s


In [12]:
ind_results

[{'cleaners': [{'Imputation method': AutoGluonImputation,
    'Outlier detection method': PyODKNN,
    'PPP score with cleaning': {'classification_report': {'1': {'f1-score': 0.9565217391304348,
       'precision': 0.9166666666666666,
       'recall': 1.0,
       'support': 11},
      '2': {'f1-score': 0.9818181818181818,
       'precision': 1.0,
       'recall': 0.9642857142857143,
       'support': 28},
      'accuracy': 0.9743589743589743,
      'macro avg': {'f1-score': 0.9691699604743083,
       'precision': 0.9583333333333333,
       'recall': 0.9821428571428572,
       'support': 39},
      'weighted avg': {'f1-score': 0.9746832877267659,
       'precision': 0.9764957264957264,
       'recall': 0.9743589743589743,
       'support': 39}},
     'roc_auc_acore': 0.9902597402597403}}],
  'corruptions': defaultdict(list,
              {('V9',): [MissingValues: {'column': 'V9', 'fraction': 0.426920161499486, 'na_value': nan, 'missingness': 'MAR'}]}),
  'dataset': 'parkinsons',
  'mode

In [13]:
# %%time
# for _ in range(1):
#   print("\n\n..................................ITERATION..................................\n")
#   results = []

#   for dataset in datasets:
#       for learner, param_grid in models.items():
#           for fraction in fractions:
#               results.append(run_experiment(dataset, learner, param_grid, corruptions, fraction, cleaners, 5))

In [14]:
# results